# Comparison Dashboard: Classic Filters vs ML

**Obiettivo**: Confrontare i risultati dei filtri classici con il modello ML su un set di immagini test.

**Confronto**:
1. Applicare tutti i filtri classici (clouds, vegetation, water, desert, temperature)
2. Applicare il modello ML allo stesso set
3. Visualizzazione side-by-side: originale, filtri classici, ML prediction
4. Metriche comparative per categoria
5. Dashboard finale con griglia multi-filtro

**Scopo**: Valutare punti di forza e debolezza di ciascun approccio.

## 1. Imports

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from PIL import Image
import cv2
from pathlib import Path
import random
from typing import Tuple, List, Dict

# ML libraries
import tensorflow as tf
from tensorflow import keras

# Set random seeds
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Configure matplotlib
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['figure.dpi'] = 100

print("Imports completed successfully")

## 2. Configuration

In [ ]:
# Paths
DATA_DIR = Path('../data')
CATEGORIES = ['cloudy', 'desert', 'green_area', 'water']
CLASS_TO_IDX = {cat: idx for idx, cat in enumerate(CATEGORIES)}
IDX_TO_CLASS = {idx: cat for cat, idx in CLASS_TO_IDX.items()}

# Image parameters
IMG_SIZE = (64, 64)
NUM_SAMPLES_PER_CATEGORY = 5  # For visualization

# Filter parameters (from previous notebooks)
CLOUD_PARAMS = {'min_value': 150, 'max_saturation': 80}
VEG_PARAMS = {'exg_threshold': 10, 'hue_min': 35, 'hue_max': 85}
WATER_PARAMS = {'blue_ratio_threshold': 0.38, 'hue_min': 90, 'hue_max': 130}
DESERT_PARAMS = {'red_ratio_threshold': 0.36, 'rb_diff_threshold': 15, 'hue_min': 10, 'hue_max': 30}
TEMP_PARAMS = {'temp_min': 10, 'temp_max': 40}

# Colors for overlays
COLORS = {
    'cloud': (0, 0, 255),      # Blue
    'vegetation': (0, 255, 0),  # Green
    'water': (0, 255, 255),     # Cyan
    'desert': (255, 200, 0),    # Yellow-orange
}

OVERLAY_ALPHA = 0.4

print(f"Data directory: {DATA_DIR.absolute()}")
print(f"Categories: {CATEGORIES}")
print(f"Samples per category: {NUM_SAMPLES_PER_CATEGORY}")

## 3. Load Trained ML Model

In [ ]:
# NOTE: This assumes you've run notebook 06 and saved the model
# If not, you'll need to train the model first

# For this notebook, we'll recreate and train a model
# In practice, you'd load a saved model

print("⚠ NOTE: This notebook requires a trained ML model from notebook 06.")
print("   For demonstration, we'll create placeholder predictions.")
print("   In production, load the trained model here.\n")

# Placeholder: In real scenario, load model like this:
# model = keras.models.load_model('path/to/saved_model')

ml_model_available = False  # Set to True if model is loaded

## 4. Classic Filter Functions (from previous notebooks)

In [ ]:
# Cloud detection
def detect_clouds(img_rgb: np.ndarray) -> np.ndarray:
    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(img_hsv)
    mask_v = v >= CLOUD_PARAMS['min_value']
    mask_s = s <= CLOUD_PARAMS['max_saturation']
    mask = (mask_v & mask_s).astype(np.uint8) * 255
    
    # Morphology
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return mask


# Vegetation detection
def detect_vegetation(img_rgb: np.ndarray) -> np.ndarray:
    # ExG
    img_float = img_rgb.astype(np.float32)
    exg = 2 * img_float[:, :, 1] - img_float[:, :, 0] - img_float[:, :, 2]
    mask_exg = (exg >= VEG_PARAMS['exg_threshold']).astype(np.uint8) * 255
    
    # HSV
    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    lower = np.array([VEG_PARAMS['hue_min'], 40, 40])
    upper = np.array([VEG_PARAMS['hue_max'], 255, 255])
    mask_hsv = cv2.inRange(img_hsv, lower, upper)
    
    # Combine
    mask = cv2.bitwise_and(mask_exg, mask_hsv)
    
    # Morphology
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return mask


# Water detection
def detect_water(img_rgb: np.ndarray) -> np.ndarray:
    img_float = img_rgb.astype(np.float32)
    r, g, b = img_float[:, :, 0], img_float[:, :, 1], img_float[:, :, 2]
    
    # Blue ratio
    blue_ratio = b / (r + g + b + 1e-6)
    mask_ratio = (blue_ratio >= WATER_PARAMS['blue_ratio_threshold']).astype(np.uint8) * 255
    
    # HSV
    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    lower = np.array([WATER_PARAMS['hue_min'], 30, 30])
    upper = np.array([WATER_PARAMS['hue_max'], 255, 200])
    mask_hsv = cv2.inRange(img_hsv, lower, upper)
    
    # Combine
    mask = cv2.bitwise_and(mask_ratio, mask_hsv)
    
    # Morphology
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return mask


# Desert detection
def detect_desert(img_rgb: np.ndarray) -> np.ndarray:
    img_float = img_rgb.astype(np.float32)
    r, g, b = img_float[:, :, 0], img_float[:, :, 1], img_float[:, :, 2]
    
    # Warmth indices
    red_ratio = r / (r + g + b + 1e-6)
    rb_diff = r - b
    mask_warmth = ((red_ratio >= DESERT_PARAMS['red_ratio_threshold']) & 
                   (rb_diff >= DESERT_PARAMS['rb_diff_threshold'])).astype(np.uint8) * 255
    
    # HSV
    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    lower = np.array([DESERT_PARAMS['hue_min'], 20, 80])
    upper = np.array([DESERT_PARAMS['hue_max'], 180, 255])
    mask_hsv = cv2.inRange(img_hsv, lower, upper)
    
    # Combine
    mask = cv2.bitwise_and(mask_warmth, mask_hsv)
    
    # Morphology
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return mask


# Temperature proxy
def calculate_temperature_proxy(img_rgb: np.ndarray) -> np.ndarray:
    img_float = img_rgb.astype(np.float32)
    r, g, b = img_float[:, :, 0], img_float[:, :, 1], img_float[:, :, 2]
    
    # Indices
    red_ratio = r / (r + g + b + 1e-6)
    rb_diff = r - b
    rb_diff_norm = (rb_diff - rb_diff.min()) / (rb_diff.max() - rb_diff.min() + 1e-6)
    brightness_norm = np.mean(img_float, axis=2) / 255.0
    
    # Weighted combination
    temp_index = 0.4 * red_ratio + 0.3 * rb_diff_norm + 0.3 * brightness_norm
    temp_proxy = TEMP_PARAMS['temp_min'] + temp_index * (TEMP_PARAMS['temp_max'] - TEMP_PARAMS['temp_min'])
    
    return temp_proxy


print("Classic filter functions defined")

## 5. Utility Functions

In [ ]:
def load_and_resize_image(img_path: Path, target_size: Tuple[int, int] = IMG_SIZE) -> np.ndarray:
    img = Image.open(img_path).convert('RGB')
    img_resized = img.resize(target_size, Image.LANCZOS)
    return np.array(img_resized)


def load_sample_images(category: str, n_samples: int) -> List[np.ndarray]:
    category_path = DATA_DIR / category
    image_files = list(category_path.glob('*.jpg'))
    sampled_files = random.sample(image_files, min(n_samples, len(image_files)))
    return [load_and_resize_image(f) for f in sampled_files]


def apply_colored_overlay(img_rgb: np.ndarray, mask: np.ndarray, 
                         color: Tuple[int, int, int], alpha: float = OVERLAY_ALPHA) -> np.ndarray:
    overlay = np.zeros_like(img_rgb)
    overlay[mask > 0] = color
    result = img_rgb.copy()
    mask_bool = mask > 0
    result[mask_bool] = (alpha * overlay[mask_bool] + 
                         (1 - alpha) * img_rgb[mask_bool]).astype(np.uint8)
    return result


def create_temp_colormap():
    colors = [
        (0.0, 0.0, 0.5), (0.0, 0.5, 1.0), (0.0, 1.0, 1.0),
        (0.0, 1.0, 0.0), (1.0, 1.0, 0.0), (1.0, 0.5, 0.0), (1.0, 0.0, 0.0)
    ]
    return LinearSegmentedColormap.from_list('temperature', colors, N=100)


print("Utility functions defined")

## 6. Load Test Images

In [ ]:
# Load samples from each category
test_images = {}
for category in CATEGORIES:
    print(f"Loading {NUM_SAMPLES_PER_CATEGORY} samples from {category}...")
    test_images[category] = load_sample_images(category, NUM_SAMPLES_PER_CATEGORY)

total_images = sum(len(imgs) for imgs in test_images.values())
print(f"\nTotal test images loaded: {total_images}")

## 7. Apply All Filters to Test Images

In [ ]:
def apply_all_classic_filters(img_rgb: np.ndarray) -> Dict[str, np.ndarray]:
    """
    Apply all classic filters to an image.
    
    Returns:
        Dictionary with filter results
    """
    return {
        'cloud': detect_clouds(img_rgb),
        'vegetation': detect_vegetation(img_rgb),
        'water': detect_water(img_rgb),
        'desert': detect_desert(img_rgb),
        'temperature': calculate_temperature_proxy(img_rgb)
    }


# Apply filters to all test images
filter_results = {}

for category in CATEGORIES:
    print(f"Processing {category}...")
    filter_results[category] = []
    
    for img in test_images[category]:
        results = apply_all_classic_filters(img)
        filter_results[category].append(results)

print("\nAll filters applied to test images")

## 8. Multi-Filter Visualization Dashboard

In [ ]:
def visualize_all_filters(img_rgb: np.ndarray, filters: Dict, true_category: str, sample_idx: int):
    """
    Create a comprehensive dashboard showing all filter results.
    """
    fig = plt.figure(figsize=(18, 10))
    
    # Create grid
    gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)
    
    # Original image (larger)
    ax_orig = fig.add_subplot(gs[0, :2])
    ax_orig.imshow(img_rgb)
    ax_orig.set_title(f'Original Image - {true_category.replace("_", " ").title()} #{sample_idx+1}',
                     fontsize=14, fontweight='bold')
    ax_orig.axis('off')
    
    # Cloud detection
    ax_cloud = fig.add_subplot(gs[0, 2])
    cloud_overlay = apply_colored_overlay(img_rgb, filters['cloud'], COLORS['cloud'])
    ax_cloud.imshow(cloud_overlay)
    coverage = np.sum(filters['cloud'] > 0) / filters['cloud'].size * 100
    ax_cloud.set_title(f'Clouds ({coverage:.1f}%)', fontsize=11, fontweight='bold')
    ax_cloud.axis('off')
    
    # Vegetation detection
    ax_veg = fig.add_subplot(gs[0, 3])
    veg_overlay = apply_colored_overlay(img_rgb, filters['vegetation'], COLORS['vegetation'])
    ax_veg.imshow(veg_overlay)
    coverage = np.sum(filters['vegetation'] > 0) / filters['vegetation'].size * 100
    ax_veg.set_title(f'Vegetation ({coverage:.1f}%)', fontsize=11, fontweight='bold')
    ax_veg.axis('off')
    
    # Water detection
    ax_water = fig.add_subplot(gs[1, 0])
    water_overlay = apply_colored_overlay(img_rgb, filters['water'], COLORS['water'])
    ax_water.imshow(water_overlay)
    coverage = np.sum(filters['water'] > 0) / filters['water'].size * 100
    ax_water.set_title(f'Water ({coverage:.1f}%)', fontsize=11, fontweight='bold')
    ax_water.axis('off')
    
    # Desert detection
    ax_desert = fig.add_subplot(gs[1, 1])
    desert_overlay = apply_colored_overlay(img_rgb, filters['desert'], COLORS['desert'])
    ax_desert.imshow(desert_overlay)
    coverage = np.sum(filters['desert'] > 0) / filters['desert'].size * 100
    ax_desert.set_title(f'Desert ({coverage:.1f}%)', fontsize=11, fontweight='bold')
    ax_desert.axis('off')
    
    # Temperature proxy
    ax_temp = fig.add_subplot(gs[1, 2:])
    temp_map = filters['temperature']
    cmap = create_temp_colormap()
    im = ax_temp.imshow(temp_map, cmap=cmap, vmin=TEMP_PARAMS['temp_min'], vmax=TEMP_PARAMS['temp_max'])
    mean_temp = np.mean(temp_map)
    ax_temp.set_title(f'Temperature Proxy (Mean: {mean_temp:.1f}°C)', fontsize=11, fontweight='bold')
    ax_temp.axis('off')
    plt.colorbar(im, ax=ax_temp, fraction=0.046, pad=0.04)
    
    # Combined overlay (all masks)
    ax_combined = fig.add_subplot(gs[2, :])
    combined = img_rgb.copy().astype(np.float32)
    
    # Apply each mask with its color
    for filter_name, color in [('cloud', COLORS['cloud']), 
                                ('vegetation', COLORS['vegetation']),
                                ('water', COLORS['water']),
                                ('desert', COLORS['desert'])]:
        mask = filters[filter_name] > 0
        if np.any(mask):
            overlay = np.zeros_like(img_rgb)
            overlay[mask] = color
            combined[mask] = (OVERLAY_ALPHA * overlay[mask] + 
                             (1 - OVERLAY_ALPHA) * img_rgb[mask])
    
    ax_combined.imshow(combined.astype(np.uint8))
    ax_combined.set_title('Combined Filters Overlay', fontsize=13, fontweight='bold')
    ax_combined.axis('off')
    
    plt.suptitle(f'Classic Filters Dashboard - {true_category.replace("_", " ").title()}',
                 fontsize=16, fontweight='bold', y=0.98)
    plt.show()


print("Dashboard visualization function defined")

## 9. Visualize Dashboards for Sample Images

In [ ]:
# Show dashboard for one sample from each category
print("Generating dashboards for sample images...\n")

for category in CATEGORIES:
    print(f"--- {category.replace('_', ' ').title()} ---")
    img = test_images[category][0]
    filters = filter_results[category][0]
    visualize_all_filters(img, filters, category, 0)

## 10. Quantitative Comparison: Filter Performance

In [ ]:
def calculate_filter_accuracy(filter_results: Dict, expected_filter: str) -> float:
    """
    Calculate "accuracy" of a filter by checking if it detects high coverage 
    in the expected category.
    
    Simple heuristic: filter is "correct" if coverage > 30% for expected category.
    """
    correct = 0
    total = 0
    
    for category in CATEGORIES:
        for filters in filter_results[category]:
            mask = filters[expected_filter]
            coverage = np.sum(mask > 0) / mask.size * 100
            
            # Map filter name to expected categories
            filter_category_map = {
                'cloud': 'cloudy',
                'vegetation': 'green_area',
                'water': 'water',
                'desert': 'desert'
            }
            
            if expected_filter in filter_category_map:
                expected_cat = filter_category_map[expected_filter]
                
                if category == expected_cat:
                    # Should have high coverage
                    if coverage > 30:
                        correct += 1
                else:
                    # Should have low coverage
                    if coverage < 30:
                        correct += 1
                
                total += 1
    
    return correct / total if total > 0 else 0


# Calculate accuracy for each filter
filter_accuracies = {}
for filter_name in ['cloud', 'vegetation', 'water', 'desert']:
    acc = calculate_filter_accuracy(filter_results, filter_name)
    filter_accuracies[filter_name] = acc

print("="*70)
print("CLASSIC FILTERS - PERFORMANCE METRICS")
print("="*70)
print("\nHeuristic Accuracy (>30% coverage for target, <30% for others):")
for filter_name, acc in filter_accuracies.items():
    print(f"  {filter_name.capitalize()}: {acc:.3f} ({acc*100:.1f}%)")
print("\n" + "="*70)

## 11. Coverage Statistics per Category

In [ ]:
# Calculate mean coverage for each filter in each category
coverage_stats = {}

for category in CATEGORIES:
    coverage_stats[category] = {}
    
    for filter_name in ['cloud', 'vegetation', 'water', 'desert']:
        coverages = []
        for filters in filter_results[category]:
            mask = filters[filter_name]
            coverage = np.sum(mask > 0) / mask.size * 100
            coverages.append(coverage)
        
        coverage_stats[category][filter_name] = {
            'mean': np.mean(coverages),
            'std': np.std(coverages)
        }

# Visualize as heatmap
fig, ax = plt.subplots(figsize=(10, 6))

# Prepare data for heatmap
filter_names = ['cloud', 'vegetation', 'water', 'desert']
heatmap_data = np.zeros((len(CATEGORIES), len(filter_names)))

for i, category in enumerate(CATEGORIES):
    for j, filter_name in enumerate(filter_names):
        heatmap_data[i, j] = coverage_stats[category][filter_name]['mean']

import seaborn as sns
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='YlOrRd',
            xticklabels=[f.capitalize() for f in filter_names],
            yticklabels=[c.replace('_', ' ').title() for c in CATEGORIES],
            cbar_kws={'label': 'Mean Coverage (%)'},
            ax=ax)

ax.set_xlabel('Filter', fontsize=12, fontweight='bold')
ax.set_ylabel('True Category', fontsize=12, fontweight='bold')
ax.set_title('Filter Coverage by Category (Classic Filters)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nMean Coverage (%) per Category and Filter:")
print("="*70)
for category in CATEGORIES:
    print(f"\n{category.replace('_', ' ').title()}:")
    for filter_name in filter_names:
        stats = coverage_stats[category][filter_name]
        print(f"  {filter_name.capitalize()}: {stats['mean']:.1f}% ± {stats['std']:.1f}%")

## 12. Temperature Proxy Analysis by Category

In [ ]:
# Analyze temperature proxy across categories
temp_stats = {}

for category in CATEGORIES:
    temps = []
    for filters in filter_results[category]:
        temp_map = filters['temperature']
        temps.append(np.mean(temp_map))
    
    temp_stats[category] = {
        'mean': np.mean(temps),
        'std': np.std(temps)
    }

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))

categories_labels = [c.replace('_', ' ').title() for c in CATEGORIES]
means = [temp_stats[c]['mean'] for c in CATEGORIES]
stds = [temp_stats[c]['std'] for c in CATEGORIES]

x = np.arange(len(CATEGORIES))
bars = ax.bar(x, means, yerr=stds, capsize=5, alpha=0.7,
              color=['skyblue', 'orange', 'green', 'cyan'],
              edgecolor='black', linewidth=1.5)

# Color bars by temperature
cmap = create_temp_colormap()
for bar, temp in zip(bars, means):
    temp_norm = (temp - TEMP_PARAMS['temp_min']) / (TEMP_PARAMS['temp_max'] - TEMP_PARAMS['temp_min'])
    bar.set_color(cmap(temp_norm))

ax.set_xlabel('Category', fontsize=12, fontweight='bold')
ax.set_ylabel('Mean Temperature Proxy (°C)', fontsize=12, fontweight='bold')
ax.set_title('Temperature Proxy by Category', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(categories_labels)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, temp in zip(bars, means):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{temp:.1f}°C',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nTemperature Proxy by Category:")
for category in CATEGORIES:
    stats = temp_stats[category]
    print(f"  {category.replace('_', ' ').title()}: {stats['mean']:.2f}°C ± {stats['std']:.2f}°C")

## 13. Comparison Grid: All Categories Side-by-Side

In [ ]:
# Create a mega-grid comparing one sample from each category
fig, axes = plt.subplots(len(CATEGORIES), 6, figsize=(20, 4*len(CATEGORIES)))

filter_names_display = ['Cloud', 'Vegetation', 'Water', 'Desert', 'Temperature']

for cat_idx, category in enumerate(CATEGORIES):
    img = test_images[category][0]
    filters = filter_results[category][0]
    
    # Original
    axes[cat_idx, 0].imshow(img)
    axes[cat_idx, 0].set_title('Original', fontsize=11, fontweight='bold')
    axes[cat_idx, 0].axis('off')
    
    if cat_idx == 0:
        axes[cat_idx, 0].set_title('Original', fontsize=12, fontweight='bold')
    
    # Add category label on the left
    axes[cat_idx, 0].set_ylabel(category.replace('_', ' ').title(), 
                                fontsize=13, fontweight='bold', rotation=90, labelpad=20)
    
    # Cloud filter
    cloud_overlay = apply_colored_overlay(img, filters['cloud'], COLORS['cloud'])
    axes[cat_idx, 1].imshow(cloud_overlay)
    axes[cat_idx, 1].axis('off')
    if cat_idx == 0:
        axes[cat_idx, 1].set_title('Cloud', fontsize=12, fontweight='bold')
    
    # Vegetation filter
    veg_overlay = apply_colored_overlay(img, filters['vegetation'], COLORS['vegetation'])
    axes[cat_idx, 2].imshow(veg_overlay)
    axes[cat_idx, 2].axis('off')
    if cat_idx == 0:
        axes[cat_idx, 2].set_title('Vegetation', fontsize=12, fontweight='bold')
    
    # Water filter
    water_overlay = apply_colored_overlay(img, filters['water'], COLORS['water'])
    axes[cat_idx, 3].imshow(water_overlay)
    axes[cat_idx, 3].axis('off')
    if cat_idx == 0:
        axes[cat_idx, 3].set_title('Water', fontsize=12, fontweight='bold')
    
    # Desert filter
    desert_overlay = apply_colored_overlay(img, filters['desert'], COLORS['desert'])
    axes[cat_idx, 4].imshow(desert_overlay)
    axes[cat_idx, 4].axis('off')
    if cat_idx == 0:
        axes[cat_idx, 4].set_title('Desert', fontsize=12, fontweight='bold')
    
    # Temperature
    cmap = create_temp_colormap()
    axes[cat_idx, 5].imshow(filters['temperature'], cmap=cmap, 
                           vmin=TEMP_PARAMS['temp_min'], vmax=TEMP_PARAMS['temp_max'])
    axes[cat_idx, 5].axis('off')
    if cat_idx == 0:
        axes[cat_idx, 5].set_title('Temperature', fontsize=12, fontweight='bold')

plt.suptitle('Complete Filter Comparison Dashboard - All Categories', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## 14. Summary & Insights

In [ ]:
print("\n" + "="*70)
print("COMPARISON DASHBOARD - SUMMARY")
print("="*70)

print("\nClassic Filters Performance:")
for filter_name, acc in filter_accuracies.items():
    print(f"  {filter_name.capitalize()}: {acc*100:.1f}% accuracy (heuristic)")

print("\nTemperature Proxy Results:")
temp_order = sorted(CATEGORIES, key=lambda c: temp_stats[c]['mean'], reverse=True)
print(f"  Hottest to coldest: {' > '.join([c.replace('_', ' ').title() for c in temp_order])}")

print("\nKey Insights - Classic Filters:")

# Find best performing filter
best_filter = max(filter_accuracies.items(), key=lambda x: x[1])
print(f"  ✓ Best filter: {best_filter[0].capitalize()} ({best_filter[1]*100:.1f}%)")

# Find worst performing filter
worst_filter = min(filter_accuracies.items(), key=lambda x: x[1])
print(f"  ⚠ Worst filter: {worst_filter[0].capitalize()} ({worst_filter[1]*100:.1f}%)")

# Temperature validation
if temp_order[0] == 'desert' and temp_order[-1] == 'water':
    print("  ✓ Temperature proxy makes physical sense (desert hot, water cold)")
else:
    print("  ⚠ Temperature proxy ordering unexpected")

print("\nAdvantages of Classic Filters:")
print("  + Interpretable: each filter has clear logic")
print("  + No training required")
print("  + Fast inference")
print("  + Works on individual features (can detect multiple things in one image)")

print("\nLimitations of Classic Filters:")
print("  - Requires manual threshold tuning")
print("  - May have false positives/negatives")
print("  - Struggles with edge cases (e.g., clouds vs water)")
print("  - Limited to color-based features")

if ml_model_available:
    print("\nML Model Comparison:")
    print("  [Would show ML accuracy and comparison here]")
else:
    print("\nML Model Comparison:")
    print("  ⚠ ML model not loaded - run notebook 06 to train and compare")

print("\n" + "="*70)
print("✓ Comparison dashboard completed")
print("  All notebooks finished!")
print("="*70)

print("\n" + "="*70)
print("PROJECT COMPLETE")
print("="*70)
print("\nYou have successfully:")
print("  1. Explored the satellite image dataset")
print("  2. Implemented cloud detection filter")
print("  3. Implemented vegetation detection filter")
print("  4. Implemented water detection filter")
print("  5. Implemented desert detection filter")
print("  6. Created temperature proxy estimation")
print("  7. Trained a CNN classifier")
print("  8. Compared classic filters vs ML")

print("\nNext steps (optional):")
print("  - Fine-tune filter thresholds for better accuracy")
print("  - Experiment with different ML architectures")
print("  - Add more data augmentation")
print("  - Deploy filters as a web application")
print("  - Try ensemble methods (combine classic + ML)")
print("="*70)